In [ ]:
import os
import sys
import base64
from io import BytesIO

import numpy as np
from PIL import Image

sys.path.append("..")
from dash_reusable_components import *

# Displays images smaller
def display(im, new_width=400):
    ratio = new_width / im.size[0]
    new_height = round(im.size[1] * ratio)
    return im.resize((new_width, new_height))

## Testing PIL vs b64

In [ ]:
image_path = "../images/IU.jpg"

im = Image.open(image_path)
print("Shape of Image:", im.size)
print("Size of Image:", os.stat(image_path).st_size, "bytes")
display(im)

### Encoding

In [ ]:
enc_png = pil_to_b64(im)
print("PNG results:")
print("Length of string:", len(enc_png))
print("Size of string:", sys.getsizeof(enc_png), "bytes")
print("Time taken to convert from PIL to b64:")
%timeit pil_to_b64(im)

enc_jpg = pil_to_b64(im, enc_format='jpeg')
print("\nJPEG results:")
print("Length of string:", len(enc_jpg))
print("Size of string:", sys.getsizeof(enc_jpg), "bytes")
print("Time taken to convert from PIL to b64:")
%timeit pil_to_b64(im, enc_format='jpeg')

### Decoding

In [ ]:
dec_png = b64_to_pil(enc_png)
print("PNG results:")
print("Time taken to convert from b64 to PIL:")
%timeit b64_to_pil(enc_png)

dec_jpg = b64_to_pil(enc_jpg)
print("\nJPEG results:")
print("Time taken to convert from b64 to PIL:")
%timeit b64_to_pil(enc_jpg)

In [ ]:
decoded = b64_to_pil(enc_png)
display(decoded)

## Testing Numpy and b64

### Encoding

In [ ]:
# Get numpy array from previous image
np_array = np.asarray(im)
print("Numpy array shape:", np_array.shape)
print("Numpy array size:", np_array.nbytes, "bytes")

enc_png = numpy_to_b64(im, scalar=False, enc_format='png')
print("\nPNG results:")
print("Length of string:", len(enc_png))
print("Size of string:", sys.getsizeof(enc_png), "bytes")
print("Time taken to convert from Numpy to b64:")
%timeit numpy_to_b64(im, scalar=False)

enc_jpg = numpy_to_b64(im, scalar=False, enc_format='jpeg')
print("\nJPEG results:")
print("Length of string:", len(enc_jpg))
print("Size of string:", sys.getsizeof(enc_jpg), "bytes")
print("Time taken to convert from Numpy to b64:")
%timeit numpy_to_b64(im, scalar=False, enc_format='jpeg')

### Decoding

In [ ]:
dec_png = b64_to_numpy(enc_png, to_scalar=False)
print("PNG results:")
print("Time taken to convert from b64 to Numpy:")
%timeit b64_to_numpy(enc_png)
print("Time taken to convert from b64 to Numpy (to_scalar false):")
%timeit b64_to_numpy(enc_png, to_scalar=False)


dec_jpg = b64_to_numpy(enc_jpg, to_scalar=False)
print("\nJPEG results:")
print("Time taken to convert from b64 to Numpy:")
%timeit b64_to_numpy(enc_jpg)
print("Time taken to convert from b64 to Numpy (to_scalar false):")
%timeit b64_to_numpy(enc_jpg, to_scalar=False)

## Testing PIL and Bytes Encoding/Decoding

In [ ]:
print("Time taken to convert from PIL to bytes string:")
%timeit pil_to_bytes_string(im)

enc_b, im_size, mode = pil_to_bytes_string(im)

print("\nTime taken to convert from bytes string to PIL:")
%timeit bytes_string_to_pil(enc_b, im_size)

### Compare Matching for Jpeg and png encodings

In [ ]:
print("dec_png and np_array are same:", np.all(dec_png == np_array))
print("dec_jpg and np_array are same:", np.all(dec_jpg == np_array))

matching_count = np.count_nonzero(dec_jpg == np_array)
non_matching_count = np.count_nonzero(dec_jpg != np_array)
total = matching_count + non_matching_count

print("\nNumber of matching values:", matching_count)
print("Number of non-matching values:", non_matching_count)
print(f"{100 * matching_count / total:.2f}% matching vs {100 * non_matching_count / total:.2f}% not matching")

display(Image.fromarray(dec_jpg))

## Conversion speed at different dimensions

### PIL to b64

In [ ]:
heights = [360, 480, 720, 1080, 2160]

for height in heights:
    width = round(height * 16 / 9)
    resized_im = im.resize((width, height))
    
    print(f"Size: {width}x{height}")
    print("Time taken to convert from PIL to b64 (png):")
    %timeit pil_to_b64(resized_im, enc_format='png')
    print("Time taken to convert from PIL to b64 (jpeg):")
    %timeit pil_to_b64(resized_im, enc_format='jpeg')
    print()

### Numpy to b64

In [ ]:
heights = [360, 480, 720, 1080, 2160]

for height in heights:
    width = round(height * 16 / 9)
    resized_im = im.resize((width, height))
    
    print(f"Size: {width}x{height}")
    print("Time taken to convert from numpy to b64 (png):")
    %timeit numpy_to_b64(resized_im, scalar=False)
    print("Time taken to convert from numpy to b64 (jpeg):")
    %timeit numpy_to_b64(resized_im, enc_format='jpeg', scalar=False)
    print()

In [ ]:
buff = BytesIO()
%timeit im.save(buff, format='png', compression_level=1)
%timeit encoded = base64.b64encode(buff.getvalue())

## Exploring Jpeg Compression

In [ ]:
dec_jpg.filter(ImageFilter.BLUR).size

In [ ]:
from PIL import ImageFilter

im = Image.open('../images/cats.jpg')
np_array = np.asarray(im)

for x in range(1, 11):
    enc_jpg = pil_to_b64(im, enc_format='jpeg', quality=100)
    dec_jpg = b64_to_pil(enc_jpg)
    
    random = np.random.randint(0, 1500)
    # Apply some operation
    box = (random, random, random + 50,  random + 50)
    cropped = dec_jpg.filter(ImageFilter.BLUR).crop(box)
    dec_jpg.paste(cropped, box=box)
    
    dec_arr = np.asarray(dec_jpg)
    
    matching_count = np.count_nonzero(dec_arr == np_array)
    non_matching_count = np.count_nonzero(dec_arr != np_array)
    total = matching_count + non_matching_count

    print(f"\nNumber of matching values after {x} compressions: {matching_count}")
    print("Number of non-matching values:", non_matching_count)
    print(f"{100 * matching_count / total:.2f}% matching vs {100 * non_matching_count / total:.2f}% not matching")

### Exploring Lossless jpeg compression (jpeg 2000)

In [ ]:
def pil_to_b64(im, enc_format='png', verbose=False, **kwargs):
    """
    Converts a PIL Image into base64 string for HTML displaying
    :param im: PIL Image object
    :param enc_format: The image format for displaying. If saved the image will have that extension.
    :return: base64 encoding
    """
    t_start = time.time()

    buff = BytesIO()
    im.save(buff, format=enc_format, **kwargs)
    encoded = base64.b64encode(buff.getvalue()).decode("utf-8")

    t_end = time.time()
    if verbose:
        print(f"PIL converted to b64 in {t_end - t_start:.3f} sec")

    return encoded

In [ ]:
%timeit pil_to_b64(im, enc_format='png')

In [ ]:
%timeit pil_to_b64(im, enc_format='jpeg2000')

In [ ]:
%timeit pil_to_b64(im, enc_format='jpeg')

### Exploring Jpeg compression Sizes

In [ ]:
%timeit pil_to_b64(im, enc_format='jpeg', quality=100)
%timeit pil_to_b64(im, enc_format='jpeg', quality=95)

In [ ]:
im = Image.open('../images/cats.jpg')
print(len(pil_to_b64(im, enc_format='jpeg', quality=90)))
print(len(pil_to_b64(im, enc_format='jpeg', quality=95)))
print(len(pil_to_b64(im, enc_format='jpeg', quality=100)))

## Supplementary Exploration

In [ ]:
import pandas as pd
im = Image.open('../images/IU2.jpg')
arr = np.asarray(im)

print(arr.size)

%timeit im.getdata()
%timeit pil_to_b64(im)
%timeit Image.fromarray(arr)

In [ ]:
barr = arr.tobytes()
back = np.frombuffer(barr, dtype=np.uint8).reshape(arr.shape)
display(Image.fromarray(back))

In [ ]:
%timeit barr = np.asarray(im).tobytes()
%timeit Image.fromarray(np.frombuffer(barr, dtype=np.uint8).reshape(arr.shape))

In [ ]:
%timeit imgSize = im.size
%timeit rawData = im.tobytes()
%timeit Image.frombytes('RGB', imgSize, rawData)

In [ ]:
im = Image.open('../images/IU2.jpg')
imgSize = im.size
imb = im.tobytes()
enc_str = base64.b64encode(imb).decode('ascii')

dec = base64.b64decode(enc_str.encode('ascii'))
display(Image.frombytes('RGB', imgSize, dec))

In [ ]:
im = Image.open('../images/IU2.jpg')
arr = np.asarray(im)
arrb = arr.tobytes()
enc_str = base64.b64encode(barr).decode('ascii')
imgSize = arr.shape

dec = base64.b64decode(enc_str.encode('ascii'))
retrieved_arr = np.frombuffer(barr, dtype=np.uint8).reshape(imgSize)

im_retrieved = Image.fromarray(retrieved_arr)
print(type(im_retrieved))
display(im_retrieved)

In [ ]:
%timeit pil_to_b64(im, enc_format='bmp')
string = pil_to_b64(im, enc_format='bmp')
%timeit b64_to_pil(string)

In [ ]:
# Image utility functions
def pil_to_b64_png(im, verbose=False, comp=6):
    """
    Converts a PIL Image into base64 string for HTML displaying
    :param im: PIL Image object
    :param enc_format: The image format for displaying. If saved the image will have that extension.
    :return: base64 encoding
    """
    t_start = time.time()

    buff = BytesIO()
    im.save(buff, format='png', compress_level=comp)
    encoded = base64.b64encode(buff.getvalue()).decode("utf-8")

    t_end = time.time()
    if verbose:
        print(f"PIL converted to b64 in {t_end - t_start:.3f} sec")

    return encoded

%timeit pil_to_b64_png(im, comp=1)
string = pil_to_b64_png(im, comp=1)
%timeit b64_to_pil(string)

In [ ]:
def func(im):
    buff = BytesIO()
    im.save(buff, format='png', compress_level=1)
    
%timeit func(im)